In [1]:
!pip install -q flwr[simulation] torch torchvision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.1/330.1 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 49.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 MB 9.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.15.0 requires protobuf<4.21,>=3.20.3; python_version < "3.11", but you have protobuf 4.25.3 which is incompatible.


In [1]:
from collections import OrderedDict
from typing import Dict, List, Optional, Tuple

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import CIFAR10
from torchvision.datasets import CIFAR100
import time
import flwr as fl

DEVICE = torch.device("cuda")  # Try "cuda" to train on GPU
print(
    f"Training on {DEVICE} using PyTorch {torch.__version__} and Flower {fl.__version__}"
)

Training on cuda using PyTorch 1.13.1 and Flower 1.4.0


In [2]:
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
   process = psutil.Process(os.getpid())
   print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
   print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

Gen RAM Free: 123.4 GB  | Proc size: 305.2 MB
GPU RAM Free: 15127MB | Used: 1041MB | Util   6% | Total 16376MB


In [3]:
NUM_CLIENTS = 10


def load_datasets(num_clients: int):
    # Download and transform CIFAR-10 (train and test)
    transform = transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
    )
    trainset = CIFAR100("./dataset", train=True, download=True, transform=transform)
    testset = CIFAR100("./dataset", train=False, download=True, transform=transform)

    # Split training set into `num_clients` partitions to simulate different local datasets
    partition_size = len(trainset) // num_clients
    lengths = [partition_size] * num_clients
    datasets = random_split(trainset, lengths, torch.Generator().manual_seed(42))

    # Split each partition into train/val and create DataLoader
    trainloaders = []
    valloaders = []
    for ds in datasets:
        len_val = len(ds) // 10  # 10 % validation set
        len_train = len(ds) - len_val
        lengths = [len_train, len_val]
        ds_train, ds_val = random_split(ds, lengths, torch.Generator().manual_seed(42))
        trainloaders.append(DataLoader(ds_train, batch_size=32, shuffle=True))
        valloaders.append(DataLoader(ds_val, batch_size=32))
    testloader = DataLoader(testset, batch_size=32)
    return trainloaders, valloaders, testloader


trainloaders, valloaders, testloader = load_datasets(NUM_CLIENTS)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
class Net(nn.Module):
    def __init__(self) -> None:
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 100)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]


def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)


def train(net, trainloader, epochs: int):
    """Train the network on the training set."""
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters())
    net.train()
    for epoch in range(epochs):  # Use the passed 'epochs' variable here
        correct, total, epoch_loss = 0, 0, 0.0
        for images, labels in trainloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            # Metrics
            epoch_loss += loss.item()  # Make sure to call .item() to get the scalar value
            total += labels.size(0)
            correct += (torch.max(outputs.data, 1)[1] == labels).sum().item()
        epoch_loss /= len(trainloader.dataset)
        epoch_acc = correct / total
        print(f"Epoch {epoch}: train loss {epoch_loss:.6f}, accuracy {epoch_acc:.6f}")



def test(net, testloader):
    """Evaluate the network on the entire test set."""
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss /= len(testloader.dataset)
    accuracy = correct / total
    return loss, accuracy

In [5]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, valloader):
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self):
        print(f"[Client {self.cid}] get_parameters")
        return get_parameters(self.net)

    def fit(self, parameters, config):
        print(f"[Client {self.cid}] fit, config: {config}")
        set_parameters(self.net, parameters)
        epochs = config.get("epochs", 1)
        start_time = time.time()  # Start time measurement
        train(self.net, self.trainloader, epochs)
        training_time = time.time() - start_time  # Calculate duration
        print(f"Training time for Client {self.cid}: {training_time:.2f} seconds")
        return get_parameters(self.net), len(self.trainloader), {"training_time": training_time}



    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}] evaluate, config: {config}")
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}


def client_fn(cid) -> FlowerClient:
    net = Net().to(DEVICE)
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]
    return FlowerClient(cid, net, trainloader, valloader)

In [6]:
from typing import Callable, Union

from flwr.common import (
    EvaluateIns,
    EvaluateRes,
    FitIns,
    FitRes,
    MetricsAggregationFn,
    NDArrays,
    Parameters,
    Scalar,
    ndarrays_to_parameters,
    parameters_to_ndarrays,
)
from flwr.server.client_manager import ClientManager
from flwr.server.client_proxy import ClientProxy
from flwr.server.strategy.aggregate import aggregate, weighted_loss_avg


class FedCustom(fl.server.strategy.Strategy):
    def __init__(
        self,
        fraction_fit: float = 1.0,
        fraction_evaluate: float = 1.0,
        min_fit_clients: int = 2,
        min_evaluate_clients: int = 2,
        min_available_clients: int = 2,
    ) -> None:
        super().__init__()
        self.fraction_fit = fraction_fit
        self.fraction_evaluate = fraction_evaluate
        self.min_fit_clients = min_fit_clients
        self.min_evaluate_clients = min_evaluate_clients
        self.min_available_clients = min_available_clients
        self.client_training_times = {}
    def __repr__(self) -> str:
        return "FedCustom"

    def initialize_parameters(
        self, client_manager: ClientManager
    ) -> Optional[Parameters]:
        """Initialize global model parameters."""
        net = Net()
        ndarrays = get_parameters(net)
        return fl.common.ndarrays_to_parameters(ndarrays)

    def configure_fit(self, server_round: int, parameters: Parameters, client_manager: ClientManager):
        sample_size, min_num_clients = self.num_fit_clients(client_manager.num_available())
        clients = client_manager.sample(num_clients=sample_size, min_num_clients=min_num_clients)
        epochs_sc = 5
        epochs_hl = 3

        standard_config = {"lr": 0.001, "epochs": epochs_sc}
        higher_lr_config = {"lr": 0.0001, "epochs": epochs_hl}
        fit_configurations = []

        for client in clients:
            # Choose config based on the previous training time
            last_time = self.client_training_times.get(client.cid, 0)  # Default to 0 if no time recorded
            print(f"This is the last time {last_time}")
            


            config_to_use = standard_config if last_time < 13.8 else higher_lr_config
            fit_configurations.append((client, FitIns(parameters, config_to_use)))

        return fit_configurations

    def aggregate_fit(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, FitRes]],
        failures: List[Union[Tuple[ClientProxy, FitRes], BaseException]],
    ) -> Tuple[Optional[Parameters], Dict[str, Scalar]]:
        """Aggregate fit results using weighted average."""
        for client, fit_res in results:
            # Update training times for each client
            self.client_training_times[client.cid] = fit_res.metrics.get("training_time", 0)
        weights_results = [
            (parameters_to_ndarrays(fit_res.parameters), fit_res.num_examples)
            for _, fit_res in results
        ]
        parameters_aggregated = ndarrays_to_parameters(aggregate(weights_results))
        metrics_aggregated = {}
        return parameters_aggregated, metrics_aggregated


    def configure_evaluate(
        self, server_round: int, parameters: Parameters, client_manager: ClientManager
    ) -> List[Tuple[ClientProxy, EvaluateIns]]:
        """Configure the next round of evaluation."""
        if self.fraction_evaluate == 0.0:
            return []
        config = {}
        evaluate_ins = EvaluateIns(parameters, config)

        # Sample clients
        sample_size, min_num_clients = self.num_evaluation_clients(
            client_manager.num_available()
        )
        clients = client_manager.sample(
            num_clients=sample_size, min_num_clients=min_num_clients
        )

        # Return client/config pairs
        return [(client, evaluate_ins) for client in clients]

    def aggregate_evaluate(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, EvaluateRes]],
        failures: List[Union[Tuple[ClientProxy, EvaluateRes], BaseException]],
    ) -> Tuple[Optional[float], Dict[str, Scalar]]:
        """Aggregate evaluation losses using weighted average."""

        if not results:
            return None, {}

        loss_aggregated = weighted_loss_avg(
            [
                (evaluate_res.num_examples, evaluate_res.loss)
                for _, evaluate_res in results
            ]
        )
        metrics_aggregated = {}
        return loss_aggregated, metrics_aggregated

    def evaluate(
        self, server_round: int, parameters: Parameters
    ) -> Optional[Tuple[float, Dict[str, Scalar]]]:
        """Evaluate global model parameters using an evalua
        tion function."""

        # Let's assume we won't perform the global model evaluation on the server side.
        return None

    def num_fit_clients(self, num_available_clients: int) -> Tuple[int, int]:
        """Return sample size and required number of clients."""
        num_clients = int(num_available_clients * self.fraction_fit)
        return max(num_clients, self.min_fit_clients), self.min_available_clients

    def num_evaluation_clients(self, num_available_clients: int) -> Tuple[int, int]:
        """Use a fraction of available clients for evaluation."""
        num_clients = int(num_available_clients * self.fraction_evaluate)
        return max(num_clients, self.min_evaluate_clients), self.min_available_clients

In [7]:
if DEVICE.type == "cuda":
    # Use a single client to train the global model
    client_resources = {"num_gpus": .25, "num_cpus": 2} 

In [8]:

fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=10,
    config=fl.server.ServerConfig(num_rounds=3),
    strategy=FedCustom(),  # <-- pass the new strategy here
    client_resources=client_resources,

)

INFO flwr 2024-06-03 11:48:14,010 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
INFO flwr 2024-06-03 11:48:17,651 | app.py:180 | Flower VCE: Ray initialized with resources: {'node:127.0.0.1': 1.0, 'memory': 76170031719.0, 'GPU': 1.0, 'object_store_memory': 36930013593.0, 'CPU': 32.0}
INFO flwr 2024-06-03 11:48:17,653 | server.py:86 | Initializing global parameters
INFO flwr 2024-06-03 11:48:17,665 | server.py:269 | Using initial parameters provided by strategy
INFO flwr 2024-06-03 11:48:17,666 | server.py:88 | Evaluating initial parameters
INFO flwr 2024-06-03 11:48:17,668 | server.py:101 | FL starting
DEBUG flwr 2024-06-03 11:48:17,670 | server.py:218 | fit_round 1: strategy sampled 10 clients (out of 10)


This is the last time 0
This is the last time 0
This is the last time 0
This is the last time 0
This is the last time 0
This is the last time 0
This is the last time 0
This is the last time 0
This is the last time 0
This is the last time 0
 pid=6096) [Client 4] fit, config: {'lr': 0.001, 'epochs': 5}
 pid=28124) [Client 3] fit, config: {'lr': 0.001, 'epochs': 5}
 pid=12552) [Client 0] fit, config: {'lr': 0.001, 'epochs': 5}
 pid=11516) [Client 9] fit, config: {'lr': 0.001, 'epochs': 5}
 pid=28124) Epoch 0: train loss 0.142160, accuracy 0.015778
 pid=12552) Epoch 0: train loss 0.141223, accuracy 0.024222
 pid=11516) Epoch 0: train loss 0.140727, accuracy 0.023111
 pid=6096) Epoch 0: train loss 0.140986, accuracy 0.019111
 pid=12552) Epoch 1: train loss 0.130694, accuracy 0.056000
 pid=6096) Epoch 1: train loss 0.130946, accuracy 0.057556
 pid=28124) Epoch 1: train loss 0.133681, accuracy 0.045778
 pid=11516) Epoch 1: train loss 0.130109, accuracy 0.058889
 pid=12552) Epoch 2: train loss

DEBUG flwr 2024-06-03 11:49:23,602 | server.py:232 | fit_round 1 received 10 results and 0 failures
DEBUG flwr 2024-06-03 11:49:23,643 | server.py:168 | evaluate_round 1: strategy sampled 10 clients (out of 10)


 pid=28124) Epoch 4: train loss 0.117487, accuracy 0.108222
 pid=28124) Training time for Client 2: 13.27 seconds
 pid=28124) [Client 7] evaluate, config: {}
 pid=12552) [Client 6] evaluate, config: {}
 pid=6096) [Client 2] evaluate, config: {}
 pid=11516) [Client 4] evaluate, config: {}
 pid=11516) [Client 5] evaluate, config: {}
 pid=6096) [Client 9] evaluate, config: {}
 pid=6096) [Client 1] evaluate, config: {}
 pid=28124) [Client 3] evaluate, config: {}
 pid=11516) [Client 0] evaluate, config: {}


DEBUG flwr 2024-06-03 11:49:38,811 | server.py:182 | evaluate_round 1 received 10 results and 0 failures


...

 pid=6096) Epoch 4: train loss 0.093758, accuracy 0.253778
 pid=6096) Training time for Client 5: 13.61 seconds
 pid=6096) [Client 8] evaluate, config: {}
 pid=6096) [Client 2] evaluate, config: {}
 pid=6096) [Client 1] evaluate, config: {}
 pid=6096) [Client 4] evaluate, config: {}
 pid=12552) [Client 5] evaluate, config: {}
 pid=11516) [Client 9] evaluate, config: {}
 pid=11516) [Client 3] evaluate, config: {}


DEBUG flwr 2024-06-03 11:51:39,100 | server.py:182 | evaluate_round 3 received 10 results and 0 failures


 pid=6096) [Client 0] evaluate, config: {}
 pid=28124) [Client 7] evaluate, config: {}
 pid=12552) [Client 6] evaluate, config: {}


INFO flwr 2024-06-03 11:51:39,102 | server.py:147 | FL finished in 201.4332869
INFO flwr 2024-06-03 11:51:39,105 | app.py:218 | app_fit: losses_distributed [(1, 0.1361800456047058), (2, 0.11330796208381652), (3, 0.1063378854751587)]
INFO flwr 2024-06-03 11:51:39,106 | app.py:219 | app_fit: metrics_distributed_fit {}
INFO flwr 2024-06-03 11:51:39,108 | app.py:220 | app_fit: metrics_distributed {}
INFO flwr 2024-06-03 11:51:39,109 | app.py:221 | app_fit: losses_centralized []
INFO flwr 2024-06-03 11:51:39,111 | app.py:222 | app_fit: metrics_centralized {}


History (loss, distributed):
	round 1: 0.1361800456047058
	round 2: 0.11330796208381652
	round 3: 0.1063378854751587

In [9]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, valloader):
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        print(f"[Client {self.cid}] get_parameters")
        return get_parameters(self.net)

    def fit(self, parameters, config):
        print(f"[Client {self.cid}] fit, config: {config}")
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=5)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}] evaluate, config: {config}")
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        print(f"Client {self.cid} loss {loss}")
        print(f"Client {self.cid} accuracy {accuracy}")
        
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}


def client_fn(cid) -> FlowerClient:
    net = Net().to(DEVICE) #Load Model from here
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]
    return FlowerClient(cid, net, trainloader, valloader)

In [10]:
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=10,
    config=fl.server.ServerConfig(num_rounds=3),
    client_resources=client_resources,
)

INFO flwr 2024-06-03 11:51:39,224 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
INFO flwr 2024-06-03 11:51:45,620 | app.py:180 | Flower VCE: Ray initialized with resources: {'object_store_memory': 33320264908.0, 'node:127.0.0.1': 1.0, 'CPU': 32.0, 'memory': 67747284788.0, 'GPU': 1.0}
INFO flwr 2024-06-03 11:51:45,622 | server.py:86 | Initializing global parameters
INFO flwr 2024-06-03 11:51:45,625 | server.py:273 | Requesting initial parameters from one random client
INFO flwr 2024-06-03 11:51:49,701 | server.py:277 | Received initial parameters from one random client
INFO flwr 2024-06-03 11:51:49,703 | server.py:88 | Evaluating initial parameters
INFO flwr 2024-06-03 11:51:49,705 | server.py:101 | FL starting
DEBUG flwr 2024-06-03 11:51:49,707 | server.py:218 | fit_round 1: strategy sampled 10 clients (out of 10)


 pid=6172) [Client 4] get_parameters
 pid=6172) [Client 5] fit, config: {}
 pid=28584) [Client 9] fit, config: {}
 pid=21724) [Client 4] fit, config: {}
 pid=19552) [Client 3] fit, config: {}
 pid=6172) Epoch 0: train loss 0.143021, accuracy 0.017111
 pid=6172) Epoch 1: train loss 0.136429, accuracy 0.032667
 pid=28584) Epoch 0: train loss 0.142446, accuracy 0.021556
 pid=6172) Epoch 2: train loss 0.129091, accuracy 0.068222
 pid=28584) Epoch 1: train loss 0.134068, accuracy 0.049556
 pid=21724) Epoch 0: train loss 0.141831, accuracy 0.023333
 pid=19552) Epoch 0: train loss 0.141841, accuracy 0.020000
 pid=28584) Epoch 2: train loss 0.125417, accuracy 0.086889
 pid=6172) Epoch 3: train loss 0.123750, accuracy 0.090000
 pid=21724) Epoch 1: train loss 0.132081, accuracy 0.050222
 pid=19552) Epoch 1: train loss 0.131711, accuracy 0.060667
 pid=28584) Epoch 3: train loss 0.120798, accuracy 0.104222
 pid=6172) Epoch 4: train loss 0.119196, accuracy 0.108222
 pid=6172) [Client 2] fit, config

DEBUG flwr 2024-06-03 11:52:48,140 | server.py:232 | fit_round 1 received 10 results and 0 failures
WARNING flwr 2024-06-03 11:52:48,185 | fedavg.py:243 | No fit_metrics_aggregation_fn provided
DEBUG flwr 2024-06-03 11:52:48,187 | server.py:168 | evaluate_round 1: strategy sampled 10 clients (out of 10)


 pid=28584) Epoch 4: train loss 0.118335, accuracy 0.113778
 pid=28584) [Client 3] evaluate, config: {}
 pid=28584) Client 3 loss 0.14335215473175048
 pid=28584) Client 3 accuracy 0.04
 pid=6172) [Client 9] evaluate, config: {}
 pid=28584) [Client 2] evaluate, config: {}
 pid=6172) Client 9 loss 0.14374571323394775
 pid=6172) Client 9 accuracy 0.046
 pid=28584) Client 2 loss 0.14332037925720215
 pid=28584) Client 2 accuracy 0.044
 pid=21724) [Client 7] evaluate, config: {}
 pid=19552) [Client 1] evaluate, config: {}
 pid=21724) Client 7 loss 0.14303338909149169
 pid=21724) Client 7 accuracy 0.044
 pid=19552) Client 1 loss 0.14310094928741454
 pid=19552) Client 1 accuracy 0.048
 pid=6172) [Client 5] evaluate, config: {}
 pid=28584) [Client 6] evaluate, config: {}
 pid=21724) [Client 4] evaluate, config: {}
 pid=19552) [Client 8] evaluate, config: {}


DEBUG flwr 2024-06-03 11:53:02,639 | server.py:182 | evaluate_round 1 received 10 results and 0 failures


...

DEBUG flwr 2024-06-03 11:54:56,694 | server.py:232 | fit_round 3 received 10 results and 0 failures
DEBUG flwr 2024-06-03 11:54:56,734 | server.py:168 | evaluate_round 3: strategy sampled 10 clients (out of 10)


 pid=28584) Epoch 4: train loss 0.096531, accuracy 0.231778
 pid=28584) [Client 6] evaluate, config: {}
 pid=28584) Client 6 loss 0.10918901491165162
 pid=28584) Client 6 accuracy 0.174
 pid=28584) [Client 5] evaluate, config: {}
 pid=28584) Client 5 loss 0.1107897720336914
 pid=28584) Client 5 accuracy 0.19
 pid=28584) [Client 8] evaluate, config: {}
 pid=28584) Client 8 loss 0.1096801781654358
 pid=28584) Client 8 accuracy 0.184
 pid=28584) [Client 9] evaluate, config: {}
 pid=6172) [Client 1] evaluate, config: {}
 pid=6172) Client 1 loss 0.106356125831604
 pid=6172) Client 1 accuracy 0.196
 pid=28584) Client 9 loss 0.11244764471054078
 pid=28584) Client 9 accuracy 0.17
 pid=21724) [Client 4] evaluate, config: {}
 pid=21724) Client 4 loss 0.1134212589263916
 pid=21724) Client 4 accuracy 0.174
 pid=21724) [Client 3] evaluate, config: {}
 pid=6172) [Client 7] evaluate, config: {}
 pid=28584) [Client 0] evaluate, config: {}
 pid=21724) Client 3 loss 0.10644761180877685
 pid=21724) Clien

DEBUG flwr 2024-06-03 11:55:10,826 | server.py:182 | evaluate_round 3 received 10 results and 0 failures
INFO flwr 2024-06-03 11:55:10,829 | server.py:147 | FL finished in 201.1225916
INFO flwr 2024-06-03 11:55:10,832 | app.py:218 | app_fit: losses_distributed [(1, 0.14343816347122193), (2, 0.11558608713150025), (3, 0.10893442974090575)]
INFO flwr 2024-06-03 11:55:10,833 | app.py:219 | app_fit: metrics_distributed_fit {}
INFO flwr 2024-06-03 11:55:10,835 | app.py:220 | app_fit: metrics_distributed {}
INFO flwr 2024-06-03 11:55:10,836 | app.py:221 | app_fit: losses_centralized []
INFO flwr 2024-06-03 11:55:10,838 | app.py:222 | app_fit: metrics_centralized {}


History (loss, distributed):
	round 1: 0.14343816347122193
	round 2: 0.11558608713150025
	round 3: 0.10893442974090575

 pid=6172) Client 7 loss 0.10617020273208619
 pid=6172) Client 7 accuracy 0.206
 pid=28584) Client 0 loss 0.10692653942108155
 pid=28584) Client 0 accuracy 0.218
 pid=19552) Client 2 loss 0.10791594886779785
 pid=19552) Client 2 accuracy 0.196
